In [1]:
pip install tqdm


  Using cached tqdm-4.66.4-py3-none-any.whl (78 kB)


You should consider upgrading via the 'c:\AI\Python38\python.exe -m pip install --upgrade pip' command.


## 1. Imports 및 클래스 정의

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import concurrent.futures
import datetime
from tqdm import tqdm
# from sqlalchemy import create_engine



class Position:
    def __init__(self, buy_price, quantity,order,additional_buy_drop_rate,sell_profit_rate):
        self.buy_price = buy_price
        self.quantity = quantity
        self.order = order  # 포지션의 차수
        self.additional_buy_drop_rate = additional_buy_drop_rate  # 추가 매수 드랍 비율 추가
        self.sell_profit_rate = sell_profit_rate  # 매도 수익률 추가



## 2. 롤링 윈도우 계산 함수

In [3]:
def rolling_window_calculations(df):
    window_size = max(int(len(df) * 2 / 3), 1)  # 윈도우 크기를 정수형으로 설정, 최소값은 1
    window_size = min(window_size, 5 * 252)  # 윈도우 크기가 5년치를 넘지 않도록 설정
    min_periods = min(window_size, 252)  # 최소 기간 설정

    df = df.copy()
    df = df[df['PBR'] != 0]
    df['five_year_high'] = df['종가'].rolling(window=window_size, min_periods=min_periods).max()
    df['five_year_low'] = df['종가'].rolling(window=window_size, min_periods=min_periods).min()
    df['normalized_price'] = (df['종가'] - df['five_year_low']) / (df['five_year_high'] - df['five_year_low']) * 100  # 진입시기결정
    # df['five_year_avg_pbr'] = df['PBR'].rolling(window=window_size, min_periods=min_periods).mean()
    df = df.dropna(subset=['five_year_avg_pbr'])

    return df

## 3. 매수 및 매도 조건 체크 함수

In [4]:
def check_buy_sell_conditions(row, positions, capital, investment_per_split, num_splits, buy_threshold, buy_signals, sell_signals,code):
    normalized = row['normalized_price']
    # print(normalized)
    current_pbr = row['PBR']
    avg_pbr = row['five_year_avg_pbr'] # 5년 평균 PBR
    five_year_low = row['five_year_low']
    current_order = len(positions) + 1  # 현재 매수할 포지션의 차수, 수정해야할듯 

    # 변수 초기화
    additional_buy_drop_rate = 0
    sell_profit_rate = 0 
    
    # print(f'investment_per_split:{investment_per_split}')
     # 1차 진입 조건: normalized 가격이 buy_threshold보다 낮고 아직 매수하지 않았을 때
    if normalized < buy_threshold and len(positions) == 0:
        if capital >= investment_per_split:
            #차수별 수익률 계산 
            additional_buy_drop_rate = calculate_additional_buy_drop_rate(row['종가'], five_year_low, num_splits)
            # 매도 수익률 계산
            sell_profit_rate = calculate_sell_profit_rate(additional_buy_drop_rate)
            # print(buy_threshold,additional_buy_drop_rate,sell_profit_rate)
            # 종가, 수량 , 차수
            new_position = Position(row['종가'], int(investment_per_split / row['종가']), 1 ,sell_profit_rate, additional_buy_drop_rate)
            positions.append(new_position)
            capital -= row['종가']*int(investment_per_split / row['종가'])
            # print(f'샀어1조건 현금은:{capital}', f'{row.name} 매입금액:{investment_per_split} 종가:{row["종가"]} 차수별수익률:{additional_buy_drop_rate}')
            # row.name은 현재 행의 인덱스 (날짜)를 반환. # 매수 시점 기록
            buy_signals.append((row.name, row['종가']))
    
     # 추가 매수 조건: 이미 매수했고, 현재 가격이 이전 매수 가격보다 더 낮아졌을 때
    elif positions and len(positions) < num_splits and capital >= investment_per_split:
        last_position = positions[-1]  # 마지막 포지션(가장 최근 매수)
        if row['종가'] <= positions[-1].buy_price * (1 - last_position.additional_buy_drop_rate):
            new_position = Position(row['종가'], int(investment_per_split / row['종가']), len(positions) + 1,last_position.sell_profit_rate, last_position.additional_buy_drop_rate)
            positions.append(new_position)
            capital -= row['종가']*int(investment_per_split / row['종가'])
            # print(f'샀어2조건 현금은:{capital}', f'{row.name} 매입금액:{investment_per_split} 종가:{row["종가"]},차수별수익률:{last_position.additional_buy_drop_rate}')
            buy_signals.append((row.name, row['종가']))

   

    # Sell condition for orders 2nd to 10th
    for position in positions:
        if row['종가'] >= position.buy_price * (1 + position.sell_profit_rate ) and position.order > 1:  # 2차이면서 매수가보다 일정비율이상되면
            # print(f'매수단가: {position.buy_price} sell_profit_rate:{position.sell_profit_rate}') 
            capital += position.quantity * row['종가']
            positions.remove(position)
            sell_signals.append((row.name, row['종가'])) # 매도신호(날짜,단가) 저장
            
            # print(f'2~10차 팔았어 현금은: {capital} ,{row.name}, 종가:{row["종가"]}') 
    
    # 청산된 종목 확인을 위한 변수
    liquidated = False
    
    # Final sell signal (liquidation) for 1st order 청산
    for position in positions:
        if current_pbr >= avg_pbr and row['종가'] >= position.buy_price * (1 + position.sell_profit_rate ) and position.order == 1:
            capital += position.quantity * row['종가']
            # print(f'청산 완료 현금은: {capital}') 
            positions.remove(position)
            sell_signals.append((row.name, row['종가']))
    # print(f'이날짜 현금은 결국 : {capital},{row.name}')
            liquidated = True

    # 청산된 경우 종목 코드 반환
    if liquidated and not positions:
        return positions, capital, code
    else:
        return positions, capital, None


## 4. 매수 및 매도 수익률 계산 함수

In [5]:
def calculate_additional_buy_drop_rate(last_buy_price, five_year_low, num_splits):
    return 1 - ((five_year_low / last_buy_price) ** (1 / (num_splits - 1)))

def calculate_sell_profit_rate(buy_profit_rate):
    """
    매수 수익률을 기반으로 매도 수익률을 계산
    :buy_profit_rate: 매수 기준 수익률
    :return: 계산된 매도 수익률
    """
    sell_profit_rate = (1 / (1 - buy_profit_rate)) - 1
    return sell_profit_rate 
    

## 5. 포트폴리오 가치 업데이트 함수

In [6]:
def update_portfolio_values(positions, capital):
    portfolio_value = capital + sum(position.quantity * position.buy_price for position in positions)
    return portfolio_value

## 6. 최종 결과 계산 함수

In [7]:
def calculate_final_results(initial_capital, final_capital, total_days):
    total_profit = final_capital  - initial_capital
    return_on_investment = (total_profit / initial_capital) * 100
    total_years = total_days / 365.25
     # 예외 처리: 자본이 0 이하인 경우 CAGR 계산 방지
    if final_capital <= 0:
        cagr = -1  # 자본이 완전히 손실된 것으로 간주, CAGR을 -100%로 설정
    else:
        cagr = ((final_capital / initial_capital) ** (1 / total_years)) - 1

    return total_profit, return_on_investment, cagr # 총수익률, ROI, 연환산수익률


## 7. 종목 데이터 로드 및 기간 설정 함수

In [8]:
# 클래스 및 함수 정의 (위에서 제공된 단일 종목 백테스팅 관련 코드 사용)
def get_trading_dates(stock_data):
    # Aggregate all dates from each stock's data
    all_dates = set()
    for code, data in stock_data.items():
        all_dates.update(data.index)

    # Sort dates and return as a list
    return sorted(list(all_dates))

def load_all_stock_data(stock_codes):
    stock_data = {}
    for code in stock_codes:
        df = pd.read_excel(f'./korea_stock/{code}.xlsx')
        df['날짜'] = pd.to_datetime(df['날짜'])
        df.set_index('날짜', inplace=True)
         # 데이터 전처리
        df = rolling_window_calculations(df)
        stock_data[code] = df
    print(f'{code} done')
    return stock_data

def get_data_for_period(stock_data, start_year=2009, end_year=2023):
    period_data = {}
    for code, data in stock_data.items():
        start_date = pd.Timestamp(year=start_year, month=1, day=1)
        end_date = pd.Timestamp(year=end_year, month=12, day=31)
        period_data[code] = data[(data.index >= start_date) & (data.index <= end_date)]
    return period_data


## 8. 포트폴리오 백테스팅 함수

In [ ]:
def portfolio_backtesting(stock_codes, initial_capital, num_splits, investment_ratio,buy_threshold,stock_data):
    total_portfolio_value = initial_capital
    capital = initial_capital  # capital을 초기 자본으로 초기화
    positions_dict = {code: [] for code in stock_codes}  # 각 종목별 포지션 리스트를 위한 딕셔너리
    positions = []  # 전체 포트폴리오의 포지션
    buy_signals = []  # 매수 신호를 기록하는 리스트
    sell_signals = []  # 매도 신호를 기록하는 리스트
    portfolio_values_over_time = []
    capital_over_time = []  # 자본 변화 추적을 위한 리스트 초기화

    # 추가: 진입된 종목과 그들의 현재 최대 차수를 추적하기 위한 딕셔너리
    entered_stocks = set()  # 진입된 종목을 추적하는 집합
    current_orders_dict = {}  # 종목별 현재 차수를 추적하는 딕셔너리
       
    # print(all_trading_dates)
    previous_month = None
    
    all_trading_dates = get_trading_dates(stock_data)    
   
    # 백테스팅 실행
    for date_str  in all_trading_dates:
        # print(date_str)
        date = pd.to_datetime(date_str)# 날짜 문자열을 datetime 객체로 변환
        
        current_month = date.month
        # 새로운 달의 시작일 경우, investment_per_split 업데이트
        if current_month != previous_month:
            investment_per_split = total_portfolio_value * investment_ratio // num_splits
            # print(f'investment_per_split:{investment_per_split},total_portfolio_value : {total_portfolio_value}, investment_ratio : {investment_ratio},num_splits : {num_splits}')
            previous_month = current_month
        for code in stock_codes:
            if date in stock_data[code].index:
                row = stock_data[code].loc[date]

                if len(entered_stocks) < 7 or code in entered_stocks: # 종목수 제한 
                    positions = positions_dict[code]  # 현재 종목의 포지션 리스트
                    # print(f'종목코드{code}')
                    positions, capital ,liquidated_code = check_buy_sell_conditions(row, positions, capital, investment_per_split, num_splits, buy_threshold, buy_signals, sell_signals,code)
                    positions_dict[code] = positions  # 업데이트된 포지션 리스트를 다시 딕셔너리에 저장
                    # print('여긴가')
                    # 추가: 진입된 종목과 그들의 현재 최대 차수 업데이트
                    if positions:  # 해당 종목에 포지션이 있으면
                        entered_stocks.add(code)
                        current_order = max(position.order for position in positions)
                        current_orders_dict[code] = current_order
                    # 청산된 종목 제거
                    if liquidated_code:
                        entered_stocks.discard(liquidated_code)
                        if liquidated_code in current_orders_dict:
                            del current_orders_dict[liquidated_code]
        # 포트폴리오 가치 업데이트
        current_stock_value = 0
        for code in stock_codes:
            for position in positions_dict[code]:
                current_stock_value += position.quantity * position.buy_price
        total_portfolio_value = capital + current_stock_value
        # print(f'현금:{capital},주식가격{current_stock_value},포트폴리오가격{total_portfolio_value}')
        portfolio_values_over_time.append(total_portfolio_value)
        capital_over_time.append(capital)  # 현재 자본 추가
        # print(f"1차로 진입된 종목 개수: {len(entered_stocks)}")
        # for code, order in current_orders_dict.items():
            # print(f"종목 코드 {code}의 현재 최대 차수: {order}")
     # 백테스팅 루프 종료 후
    total_days = (all_trading_dates[-1] - all_trading_dates[0]).days
    total_years = total_days / 365.25
    final_portfolio_value = portfolio_values_over_time[-1]
    cagr = ((final_portfolio_value / initial_capital) ** (1 / total_years)) - 1
        
    return positions_dict, total_portfolio_value, portfolio_values_over_time, capital_over_time, buy_signals, sell_signals, all_trading_dates, cagr


## 9. 백테스팅 래퍼 및 평균 CAGR 계산 함수

In [ ]:
def backtesting_wrapper(params):
    num_splits, buy_threshold, investment_ratio = params
    try:
        # print(f'Starting: num_splits={num_splits}, buy_threshold={buy_threshold}, investment_ratio={investment_ratio}')
        _, _, _, _, _, _, _, cagr = portfolio_backtesting(stock_codes, initial_capital, num_splits, investment_ratio, buy_threshold, stock_data)
        # print(f'Completed: num_splits={num_splits}, buy_threshold={buy_threshold}, investment_ratio={investment_ratio}, CAGR={cagr}')
        return num_splits, buy_threshold, investment_ratio, cagr
    except Exception as e:
        print(f'Error in backtesting: {e}')
        return num_splits, buy_threshold, investment_ratio, None  # 오류가 발생한 경우 CAGR을 None으로 설정
 
 def check_if_already_calculated(num_splits, buy_threshold, investment_ratio):
    if os.path.exists(results_file):
        existing_results = pd.read_csv(results_file)
        return any((existing_results['num_splits'] == num_splits) & 
                   (existing_results['buy_threshold'] == buy_threshold) &
                   (existing_results['investment_ratio'] == investment_ratio))
    return False   

def run_backtesting_for_period(stock_codes, initial_capital, num_splits, investment_ratio, buy_threshold, start_year, end_year, loaded_stock_data):
    # 주어진 기간에 대한 데이터 로드
    period_stock_data = get_data_for_period(loaded_stock_data, start_year, end_year)
    
    # 백테스팅 수행
    _, total_portfolio_value, _, _, _, _, _, cagr = portfolio_backtesting(
        stock_codes, initial_capital, num_splits, investment_ratio, buy_threshold, loaded_stock_data)
    
    return total_portfolio_value, cagr

def calculate_average_results(backtesting_results):
    total_values = [result[0] for result in backtesting_results]
    cagr_values = [result[1] for result in backtesting_results]

    average_total_value = sum(total_values) / len(total_values)
    average_cagr = sum(cagr_values) / len(cagr_values)

    return average_total_value, average_cagr

# 시간 기간에 따른 평균 CAGR을 계산하는 래퍼 함수
def average_cagr_wrapper(params):
    num_splits, buy_threshold, investment_ratio = params
    period_results = []
    for start_year, end_year in time_periods:
        try:
            total_value, cagr = run_backtesting_for_period(
                stock_codes, initial_capital, num_splits, investment_ratio, buy_threshold, start_year, end_year,stock_data)
            period_results.append((total_value, cagr))
            print(f'기간 하나 완료{params}')
        except Exception as e:
            print(f'Error in backtesting for period {start_year}-{end_year}: {e}')
            period_results.append((0, None))  # 오류가 발생한 경우 결과를 (0, None)으로 설정

    average_results = calculate_average_results(period_results)
    return num_splits, buy_threshold, investment_ratio, average_results[0], average_results[1]


## 10. 메인 실행 코드

In [ ]:

initial_capital = 100000000  # 초기 자본
num_splits_options = [10] # 15,20,25,30
buy_threshold_options = [30, 35, 40, 45, 50, 55, 60]
investment_ratio_options = [0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

time_periods = [(2009, 2014), (2011, 2016), (2013, 2018), (2015, 2020), (2017, 2022), (2018, 2023), (2009, 2016), (2016, 2023)]

combinations = [(n, b, i) for n in num_splits_options for b in buy_threshold_options for i in investment_ratio_options]

# 포트폴리오 백테스팅 실행
stock_codes = [ 
#                '021240',  
#               '037710',
#                '010420',
#                 '016920',
# '021240',
# '032750',
# '037400',
# '037710',
# # '039740',
# '045060',
#             '075130',
#             '088790',
#             '094970',
#             '136540',
#             # '225590',
#             '317400',
              
'072870',
'005010',
'003200',
'004590',
'003690',        
            # '003230',
            '035000',
            '000850',
            '004000',
            '002920',
    '054040',
    '093380',
    '057050',
    '011780',
    '155650',
'003800',
'005180',
'010240',
'023590',
'001560',
    '092730',
    # '051600',
    # '012750',
    '033780',
    '026890',
'002960',
'031980',
# '030000',
# '057030',
'065710',
   '067920',
    '111770',
    '002150',
    '003080',
    '051390',
    '021080',
'069510',
'011560',
'091700',
'039420',
'100840',
    '009160',
    '089600',
    # '251970',
    '105840',
    # '068330',
# '134060',
'105840',
'130660',
'012690',
# '122870',
    '052600',
    '094820',
    # '115500',
    '090470',
    '056190'
    
]

# # 모든 종목 데이터 로드
stock_data = load_all_stock_data(stock_codes)

# # 결과를 DataFrame으로 변환하여 출력
# results_df = pd.DataFrame(all_results, columns=["num_splits", "buy_threshold", "investment_ratio", "Average_Total_Value", "Average_CAGR"])
# print(results_df.sort_values(by="Average_CAGR", ascending=False))

# 병렬 백테스팅 실행 및 결과 저장


In [ ]:

with concurrent.futures.ThreadPoolExecutor(max_workers=224) as executor:
    futures = [executor.submit(average_cagr_wrapper, param) for param in combinations]
    # results = [future.result() for future in concurrent.futures.as_completed(futures)]
    results = []
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(combinations)):
        result = future.result()
        results.append(result)

# 결과 DataFrame 생성 및 저장
results_df = pd.DataFrame(results, columns=["num_splits", "buy_threshold", "investment_ratio", "Average_Total_Value", "Average_CAGR"])
results_df.to_csv('average_backtesting_results_7_10split_2.csv', index=False)

# 결과 출력
# print(results_df.sort_values(by="Average_CAGR", ascending=False))